# LinkedIn PR Sentiment Classification Model

Using Gemini LLM embeddings and XGBoost to predict whether a post will result in positive or negative PR.


In [1]:
# Install required dependencies
!pip install -q google-generativeai xgboost pandas numpy scikit-learn vaderSentiment matplotlib seaborn


In [2]:
# Import libraries
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Gemini API
import google.generativeai as genai

print("✓ All libraries imported successfully")


✓ All libraries imported successfully


In [ ]:
# Configure Gemini API
# You'll need to set your API key here or as an environment variable
GEMINI_API_KEY =  # Set your API key here or export GEMINI_API_KEY="your-key"

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✓ Gemini API configured")
else:
    print("⚠ Warning: GEMINI_API_KEY not set. You'll need to set it before generating embeddings.")
    print("  Set it with: export GEMINI_API_KEY='your-api-key'")
    print("  Or in the notebook: GEMINI_API_KEY = 'your-api-key'")


✓ Gemini API configured


## 1. Load and Explore Data

### Setup for Google Colab (if needed)

**If using Google Colab**, run this cell first to upload your data files:


In [5]:
# Google Colab Setup - Upload data files (OPTIONAL)
# This cell checks if data files exist. If using Colab AND files don't exist, it will prompt for upload.
# Otherwise, it will use existing local files.

# Define filenames we need
posts_filename = 'dataset_linkedin-company-posts_2025-12-14_07-04-03-569.json'
comments_filename = 'dataset_linkedin-post-comments-replies-engagements-scraper-no-cookies_2025-12-14_04-25-32-524.json'

# Check if files already exist locally
local_paths_to_check = [
    'data/',
    '/home/micha/dev/lyra_hackathon/data/',
    os.path.join(os.getcwd(), 'data/'),
]

files_exist_locally = False
for path in local_paths_to_check:
    if os.path.exists(os.path.join(path, posts_filename)) and os.path.exists(os.path.join(path, comments_filename)):
        files_exist_locally = True
        print(f"✓ Data files found in: {path}")
        break

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔵 Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("💻 Running in local environment")

# Only prompt for upload if in Colab AND files don't exist
if IN_COLAB and not files_exist_locally:
    from google.colab import files
    print("\n📤 Data files not found. Please upload them:")
    print("   1. dataset_linkedin-company-posts_2025-12-14_07-04-03-569.json")
    print("   2. dataset_linkedin-post-comments-replies-engagements-scraper-no-cookies_2025-12-14_04-25-32-524.json")
    
    uploaded = files.upload()
    
    if not os.path.exists('/content/data'):
        os.makedirs('/content/data')
    
    for filename in uploaded.keys():
        import shutil
        shutil.move(filename, f'/content/data/{filename}')
        print(f"✓ Moved {filename} to /content/data/")
else:
    if files_exist_locally:
        print("✓ Using existing local data files - no upload needed")
    else:
        print("⚠️ Will attempt to locate files in next cell")

print("\n✓ Setup complete - proceed to next cell")


🔵 Running in Google Colab environment

📤 Data files not found. Please upload them:
   1. dataset_linkedin-company-posts_2025-12-14_07-04-03-569.json
   2. dataset_linkedin-post-comments-replies-engagements-scraper-no-cookies_2025-12-14_04-25-32-524.json


KeyboardInterrupt: 

In [ ]:
# Load data files - works for both Colab and local environments
posts_filename = 'dataset_linkedin-company-posts_2025-12-14_07-04-03-569.json'
comments_filename = 'dataset_linkedin-post-comments-replies-engagements-scraper-no-cookies_2025-12-14_04-25-32-524.json'

# Define possible paths
possible_paths = [
    '/content/data/',  # Google Colab
    'data/',  # Local relative
    '/home/micha/dev/lyra_hackathon/data/',  # Local absolute
    '/content/',  # Colab root
    ''  # Current directory
]

# Find posts file
posts_path = None
for base_path in possible_paths:
    test_path = os.path.join(base_path, posts_filename)
    if os.path.exists(test_path):
        posts_path = test_path
        break

if not posts_path:
    raise FileNotFoundError(f"Cannot find {posts_filename}. Please upload the file or check the path.")

print(f"📂 Loading posts from: {posts_path}")

# Load posts file
with open(posts_path, 'r', encoding='utf-8') as f:
    posts_raw = json.load(f)

print(f"✓ Loaded {len(posts_raw)} posts")

# Convert to DataFrame
posts_df = pd.DataFrame(posts_raw)
print(f"  Posts DataFrame shape: {posts_df.shape}")
print(f"  Columns: {list(posts_df.columns)}")


In [ ]:
# Load comments file
comments_path = None
for base_path in possible_paths:
    test_path = os.path.join(base_path, comments_filename)
    if os.path.exists(test_path):
        comments_path = test_path
        break

if not comments_path:
    raise FileNotFoundError(f"Cannot find {comments_filename}. Please upload the file or check the path.")

print(f"📂 Loading comments from: {comments_path}")

# Load comments file
with open(comments_path, 'r', encoding='utf-8') as f:
    comments_raw = json.load(f)

print(f"✓ Loaded {len(comments_raw)} comments")

# Convert to DataFrame
comments_df = pd.DataFrame(comments_raw)
print(f"  Comments DataFrame shape: {comments_df.shape}")
print(f"  Columns: {list(comments_df.columns)}")


In [ ]:
# Explore posts structure
print("=" * 60)
print("POSTS DATA EXPLORATION")
print("=" * 60)
print(f"\nFirst post sample:")
print(f"Text: {posts_df['text'].iloc[0][:200]}...")
print(f"\nPost types: {posts_df['post_type'].value_counts().to_dict()}")
print(f"\nEngagement stats:")
posts_df['stats'].iloc[0]


In [ ]:
# Flatten stats dictionary into columns
stats_df = pd.json_normalize(posts_df['stats'])
posts_df = pd.concat([posts_df.drop('stats', axis=1), stats_df], axis=1)

print("Flattened engagement columns:")
print(posts_df[['total_reactions', 'like', 'comments', 'reposts']].describe())


In [ ]:
# Flatten author and posted_at information
author_df = pd.json_normalize(posts_df['author'])
author_df.columns = ['author_' + col for col in author_df.columns]

posted_at_df = pd.json_normalize(posts_df['posted_at'])
posted_at_df.columns = ['posted_' + col for col in posted_at_df.columns]

posts_df = pd.concat([
    posts_df.drop(['author', 'posted_at'], axis=1),
    author_df,
    posted_at_df
], axis=1)

print(f"\nUpdated posts DataFrame shape: {posts_df.shape}")
print(f"Key columns: {[c for c in posts_df.columns if c in ['text', 'activity_urn', 'total_reactions', 'comments', 'reposts', 'posted_date', 'author_name']]}")


In [ ]:
# Explore comments structure
print("\n" + "=" * 60)
print("COMMENTS DATA EXPLORATION")
print("=" * 60)
print(f"\nFirst comment text: {comments_df['text'].iloc[0][:200]}...")
print(f"\nPost IDs in comments: {comments_df['post_input'].nunique()} unique posts")
print(f"Comments per post: mean={comments_df.groupby('post_input').size().mean():.1f}, median={comments_df.groupby('post_input').size().median():.0f}")


# Try to link comments to posts
# Comments use post_input field which contains activity URN
# Posts have activity_urn field

# Extract activity URN from posts for matching
posts_df['comment_match_key'] = posts_df['activity_urn']

# Check if we can match
print(f"\nSample post activity_urn: {posts_df['activity_urn'].iloc[0]}")
print(f"Sample comment post_input: {comments_df['post_input'].iloc[0]}")

# Count matches
matched_posts = posts_df[posts_df['activity_urn'].isin(comments_df['post_input'])]
print(f"\n✓ Posts with comments in dataset: {len(matched_posts)} out of {len(posts_df)}")
print(f"✓ {len(matched_posts)/len(posts_df)*100:.1f}% of posts have comments data")


In [ ]:
# Initialize sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Function to analyze comment sentiment
def analyze_comment_sentiment(text):
    """Return sentiment score from -1 (negative) to 1 (positive)"""
    if pd.isna(text) or text == '':
        return 0
    scores = sentiment_analyzer.polarity_scores(str(text))
    return scores['compound']  # compound score ranges from -1 to 1

# Test sentiment analyzer
test_texts = [
    "This is amazing! I love it!",
    "This is terrible and disappointing.",
    "Interesting post, thanks for sharing."
]
for text in test_texts:
    print(f"Text: '{text}' => Sentiment: {analyze_comment_sentiment(text):.3f}")


In [ ]:
# Calculate sentiment for all comments
print("Analyzing sentiment for all comments...")
comments_df['sentiment_score'] = comments_df['text'].apply(analyze_comment_sentiment)
print(f"✓ Sentiment analysis complete")
print(f"\nSentiment distribution:")
print(f"  Mean: {comments_df['sentiment_score'].mean():.3f}")
print(f"  Median: {comments_df['sentiment_score'].median():.3f}")
print(f"  Negative (<0): {(comments_df['sentiment_score'] < 0).sum()} ({(comments_df['sentiment_score'] < 0).sum()/len(comments_df)*100:.1f}%)")
print(f"  Neutral (0): {(comments_df['sentiment_score'] == 0).sum()} ({(comments_df['sentiment_score'] == 0).sum()/len(comments_df)*100:.1f}%)")
print(f"  Positive (>0): {(comments_df['sentiment_score'] > 0).sum()} ({(comments_df['sentiment_score'] > 0).sum()/len(comments_df)*100:.1f}%)")


In [ ]:
# Aggregate comment sentiment by post
comment_sentiment_by_post = comments_df.groupby('post_input').agg({
    'sentiment_score': ['mean', 'median', 'min', 'max', 'count'],
    'text': 'count'  # number of comments
}).reset_index()

comment_sentiment_by_post.columns = ['activity_urn', 'avg_sentiment', 'median_sentiment', 
                                       'min_sentiment', 'max_sentiment', 'sentiment_count', 'num_comments_analyzed']

print(f"Aggregated sentiment for {len(comment_sentiment_by_post)} posts")
print(f"\nSample aggregated sentiment:")
comment_sentiment_by_post.head()


In [ ]:
# Merge comment sentiment with posts
posts_with_sentiment = posts_df.merge(
    comment_sentiment_by_post,
    left_on='activity_urn',
    right_on='activity_urn',
    how='left'
)

# Fill NaN for posts without comments (assume neutral)
posts_with_sentiment['avg_sentiment'] = posts_with_sentiment['avg_sentiment'].fillna(0)
posts_with_sentiment['median_sentiment'] = posts_with_sentiment['median_sentiment'].fillna(0)
posts_with_sentiment['num_comments_analyzed'] = posts_with_sentiment['num_comments_analyzed'].fillna(0)

print(f"✓ Merged sentiment data")
print(f"Posts with sentiment data: {len(posts_with_sentiment)}")
print(f"Posts with comment sentiment: {(posts_with_sentiment['num_comments_analyzed'] > 0).sum()}")


In [ ]:
# Calculate total engagement score
posts_with_sentiment['total_engagement'] = (
    posts_with_sentiment['total_reactions'] + 
    posts_with_sentiment['comments'] * 2 +  # Comments worth more
    posts_with_sentiment['reposts'] * 3     # Reposts worth even more
)

# Calculate positive reaction ratio
positive_reactions = ['like', 'love', 'celebrate', 'support']
posts_with_sentiment['positive_reaction_sum'] = 0
for col in positive_reactions:
    if col in posts_with_sentiment.columns:
        posts_with_sentiment['positive_reaction_sum'] += posts_with_sentiment[col].fillna(0)

posts_with_sentiment['positive_reaction_ratio'] = (
    posts_with_sentiment['positive_reaction_sum'] / 
    (posts_with_sentiment['total_reactions'] + 1)  # Add 1 to avoid division by zero
)

print("Engagement metrics calculated:")
print(posts_with_sentiment[['total_engagement', 'positive_reaction_ratio', 'avg_sentiment']].describe())


In [ ]:
# Generate PR labels based on combined criteria
# Calculate thresholds
engagement_median = posts_with_sentiment['total_engagement'].median()
engagement_q25 = posts_with_sentiment['total_engagement'].quantile(0.25)
sentiment_threshold = 0.0  # Neutral point

print(f"Engagement median: {engagement_median}")
print(f"Engagement Q25: {engagement_q25}")
print(f"Sentiment threshold: {sentiment_threshold}")

# Label generation logic:
# Positive PR: High engagement (above median) AND positive sentiment (>0) AND high positive reaction ratio
# Negative PR: Low engagement (below Q25) OR negative sentiment (<-0.1) OR low positive reaction ratio

def generate_pr_label(row):
    """
    Generate PR label based on engagement and sentiment
    Returns: 1 for Positive PR, 0 for Negative PR
    """
    high_engagement = row['total_engagement'] >= engagement_median
    low_engagement = row['total_engagement'] <= engagement_q25
    positive_sentiment = row['avg_sentiment'] > 0.1  # Slightly positive
    negative_sentiment = row['avg_sentiment'] < -0.1  # Clearly negative
    good_reaction_ratio = row['positive_reaction_ratio'] > 0.85
    
    # Positive PR criteria (must meet most)
    if high_engagement and positive_sentiment and good_reaction_ratio:
        return 1  # Positive PR
    
    # Negative PR criteria (meeting any is problematic)
    if low_engagement or negative_sentiment or (row['positive_reaction_ratio'] < 0.7):
        return 0  # Negative PR
    
    # Middle ground - use engagement as tiebreaker
    return 1 if row['total_engagement'] >= engagement_median else 0

posts_with_sentiment['pr_label'] = posts_with_sentiment.apply(generate_pr_label, axis=1)

print(f"\n✓ PR labels generated")
print(f"\nLabel distribution:")
print(posts_with_sentiment['pr_label'].value_counts())
print(f"\nPercentage:")
print(posts_with_sentiment['pr_label'].value_counts(normalize=True) * 100)


In [ ]:
# Visualize label distribution with engagement
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Engagement by label
axes[0].boxplot([
    posts_with_sentiment[posts_with_sentiment['pr_label'] == 0]['total_engagement'],
    posts_with_sentiment[posts_with_sentiment['pr_label'] == 1]['total_engagement']
], labels=['Negative PR', 'Positive PR'])
axes[0].set_ylabel('Total Engagement')
axes[0].set_title('Engagement by PR Label')
axes[0].grid(True, alpha=0.3)

# Sentiment by label
axes[1].boxplot([
    posts_with_sentiment[posts_with_sentiment['pr_label'] == 0]['avg_sentiment'],
    posts_with_sentiment[posts_with_sentiment['pr_label'] == 1]['avg_sentiment']
], labels=['Negative PR', 'Positive PR'])
axes[1].set_ylabel('Average Comment Sentiment')
axes[1].set_title('Sentiment by PR Label')
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Visualization complete")


## 3. Generate Gemini Embeddings

We'll use Gemini's embedding model to convert post text into dense vector representations.


In [ ]:
# Function to generate embeddings using Gemini
def get_gemini_embedding(text, task_type="RETRIEVAL_DOCUMENT"):
    """
    Generate embedding for text using Gemini API
    """
    try:
        if not text or pd.isna(text):
            return None
        result = genai.embed_content(
            model="models/embedding-001",
            content=str(text),
            task_type=task_type
        )
        return result['embedding']
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

# Test embedding generation (only if API key is set)
if GEMINI_API_KEY:
    test_text = "This is a test post about technology and innovation."
    test_embedding = get_gemini_embedding(test_text)
    if test_embedding:
        print(f"✓ Test embedding generated successfully")
        print(f"  Embedding dimension: {len(test_embedding)}")
        print(f"  Sample values: {test_embedding[:5]}")
    else:
        print("⚠ Failed to generate test embedding")
else:
    print("⚠ Skipping embedding test - API key not set")


In [ ]:
# Generate embeddings for all posts
# Note: This will make API calls and may take a few minutes

if GEMINI_API_KEY:
    print(f"Generating embeddings for {len(posts_with_sentiment)} posts...")
    print("This may take a few minutes...")
    
    embeddings_list = []
    failed_indices = []
    
    for idx, text in enumerate(posts_with_sentiment['text']):
        if idx % 20 == 0:
            print(f"  Progress: {idx}/{len(posts_with_sentiment)}")
        
        embedding = get_gemini_embedding(text)
        if embedding:
            embeddings_list.append(embedding)
        else:
            embeddings_list.append([0] * 768)  # Fallback to zero vector
            failed_indices.append(idx)
    
    print(f"✓ Embeddings generated")
    print(f"  Success: {len(embeddings_list) - len(failed_indices)}/{len(posts_with_sentiment)}")
    if failed_indices:
        print(f"  Failed: {len(failed_indices)} posts")
    
    # Convert to numpy array
    embeddings_array = np.array(embeddings_list)
    print(f"\nEmbeddings shape: {embeddings_array.shape}")
    
    # Save embeddings to avoid re-generating
    np.save('post_embeddings.npy', embeddings_array)
    print("✓ Embeddings saved to post_embeddings.npy")
    
else:
    print("⚠ Skipping embedding generation - API key not set")
    print("  Creating placeholder embeddings for demonstration...")
    # Create random embeddings as placeholders
    embeddings_array = np.random.randn(len(posts_with_sentiment), 768)
    print(f"  Created placeholder embeddings: {embeddings_array.shape}")


## 4. Feature Engineering

Extract metadata features from posts to complement text embeddings.


In [ ]:
# Extract text-based features
def count_emojis(text):
    """Count emoji characters in text"""
    if pd.isna(text):
        return 0
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return len(emoji_pattern.findall(text))

def count_urls(text):
    """Count URLs in text"""
    if pd.isna(text):
        return 0
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return len(url_pattern.findall(str(text)))

def count_hashtags(text):
    """Count hashtags in text"""
    if pd.isna(text):
        return 0
    return len(re.findall(r'#\w+', str(text)))

def count_mentions(text):
    """Count @ mentions in text"""
    if pd.isna(text):
        return 0
    return len(re.findall(r'@\w+', str(text)))

# Apply text feature extraction
posts_with_sentiment['text_length'] = posts_with_sentiment['text'].apply(lambda x: len(str(x)) if not pd.isna(x) else 0)
posts_with_sentiment['emoji_count'] = posts_with_sentiment['text'].apply(count_emojis)
posts_with_sentiment['url_count'] = posts_with_sentiment['text'].apply(count_urls)
posts_with_sentiment['hashtag_count'] = posts_with_sentiment['text'].apply(count_hashtags)
posts_with_sentiment['mention_count'] = posts_with_sentiment['text'].apply(count_mentions)

print("✓ Text features extracted")
print(f"  Text length: mean={posts_with_sentiment['text_length'].mean():.0f}, median={posts_with_sentiment['text_length'].median():.0f}")
print(f"  Emojis: mean={posts_with_sentiment['emoji_count'].mean():.1f}")
print(f"  URLs: mean={posts_with_sentiment['url_count'].mean():.1f}")
print(f"  Hashtags: mean={posts_with_sentiment['hashtag_count'].mean():.1f}")


In [ ]:
# Extract temporal features
posts_with_sentiment['posted_datetime'] = pd.to_datetime(posts_with_sentiment['posted_date'], errors='coerce')
posts_with_sentiment['post_hour'] = posts_with_sentiment['posted_datetime'].dt.hour
posts_with_sentiment['post_day_of_week'] = posts_with_sentiment['posted_datetime'].dt.dayofweek
posts_with_sentiment['post_month'] = posts_with_sentiment['posted_datetime'].dt.month

print("✓ Temporal features extracted")
print(f"  Most common posting hour: {posts_with_sentiment['post_hour'].mode().values[0] if len(posts_with_sentiment['post_hour'].mode()) > 0 else 'N/A'}")
print(f"  Most common day of week: {posts_with_sentiment['post_day_of_week'].mode().values[0] if len(posts_with_sentiment['post_day_of_week'].mode()) > 0 else 'N/A'}")


In [ ]:
# Extract media features
posts_with_sentiment['has_media'] = posts_with_sentiment['media'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)

def get_media_type(media):
    if pd.isna(media) or media is None:
        return 'none'
    if isinstance(media, dict):
        return media.get('type', 'none')
    return 'none'

def get_media_count(media):
    if pd.isna(media) or media is None:
        return 0
    if isinstance(media, dict) and 'items' in media:
        items = media['items']
        if isinstance(items, list):
            return len(items)
    return 0

posts_with_sentiment['media_type'] = posts_with_sentiment['media'].apply(get_media_type)
posts_with_sentiment['media_count'] = posts_with_sentiment['media'].apply(get_media_count)

print("✓ Media features extracted")
print(f"  Media types: {posts_with_sentiment['media_type'].value_counts().to_dict()}")
print(f"  Average media count: {posts_with_sentiment['media_count'].mean():.2f}")


In [ ]:
# Encode categorical features
from sklearn.preprocessing import LabelEncoder

# Post type encoding
post_type_encoder = LabelEncoder()
posts_with_sentiment['post_type_encoded'] = post_type_encoder.fit_transform(posts_with_sentiment['post_type'].fillna('regular'))

# Media type encoding
media_type_encoder = LabelEncoder()
posts_with_sentiment['media_type_encoded'] = media_type_encoder.fit_transform(posts_with_sentiment['media_type'])

print("✓ Categorical features encoded")
print(f"  Post types: {dict(enumerate(post_type_encoder.classes_))}")
print(f"  Media types: {dict(enumerate(media_type_encoder.classes_))}")


In [ ]:
# Select metadata features for model
metadata_features = [
    'text_length', 'emoji_count', 'url_count', 'hashtag_count', 'mention_count',
    'post_hour', 'post_day_of_week', 'post_month',
    'has_media', 'media_count', 'media_type_encoded', 'post_type_encoded',
    'author_follower_count',
    'avg_sentiment', 'median_sentiment', 'num_comments_analyzed'
]

# Check which features exist
available_features = [f for f in metadata_features if f in posts_with_sentiment.columns]
print(f"✓ Selected {len(available_features)} metadata features:")
for feat in available_features:
    print(f"  - {feat}")

# Create metadata feature matrix
metadata_X = posts_with_sentiment[available_features].fillna(0).values
print(f"\nMetadata features shape: {metadata_X.shape}")


## 5. Combine Features and Train XGBoost Model


In [ ]:
# Combine embeddings and metadata features
print("Combining features...")
print(f"  Embeddings shape: {embeddings_array.shape}")
print(f"  Metadata shape: {metadata_X.shape}")

# Concatenate embeddings and metadata
X_combined = np.concatenate([embeddings_array, metadata_X], axis=1)
print(f"✓ Combined features shape: {X_combined.shape}")

# Get labels
y = posts_with_sentiment['pr_label'].values
print(f"  Labels shape: {y.shape}")
print(f"  Label distribution: {np.bincount(y)}")


In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Maintain class distribution
)

print("✓ Data split complete")
print(f"  Training set: {X_train.shape[0]} samples")
print(f"  Test set: {X_test.shape[0]} samples")
print(f"  Train label distribution: {np.bincount(y_train)}")
print(f"  Test label distribution: {np.bincount(y_test)}")


In [ ]:
# Scale features for better performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled")


In [ ]:
# Calculate class weights for imbalanced data
class_counts = np.bincount(y_train)
scale_pos_weight = class_counts[0] / class_counts[1] if class_counts[1] > 0 else 1.0
print(f"Class imbalance ratio: {scale_pos_weight:.2f}")


In [ ]:
# Train XGBoost classifier
print("Training XGBoost model...")

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=10
)

# Train with validation set
xgb_model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_test_scaled, y_test)],
    verbose=False
)

print("✓ Model training complete")


In [ ]:
# Make predictions
y_pred_train = xgb_model.predict(X_train_scaled)
y_pred_test = xgb_model.predict(X_test_scaled)

y_pred_proba_train = xgb_model.predict_proba(X_train_scaled)[:, 1]
y_pred_proba_test = xgb_model.predict_proba(X_test_scaled)[:, 1]

print("✓ Predictions generated")


In [ ]:
# Save the model and preprocessing objects
import joblib

joblib.dump(xgb_model, 'pr_classifier_model.pkl')
joblib.dump(scaler, 'feature_scaler.pkl')
joblib.dump(post_type_encoder, 'post_type_encoder.pkl')
joblib.dump(media_type_encoder, 'media_type_encoder.pkl')

print("✓ Model and preprocessors saved")
print("  - pr_classifier_model.pkl")
print("  - feature_scaler.pkl")
print("  - post_type_encoder.pkl")
print("  - media_type_encoder.pkl")


## 6. Model Evaluation and Interpretation


In [ ]:
# Calculate performance metrics
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

print("=" * 60)
print("MODEL PERFORMANCE METRICS")
print("=" * 60)

# Training set metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train)
train_precision, train_recall, _, _ = precision_recall_fscore_support(y_train, y_pred_train, average='binary')

print("\nTRAINING SET:")
print(f"  Accuracy:  {train_accuracy:.4f}")
print(f"  Precision: {train_precision:.4f}")
print(f"  Recall:    {train_recall:.4f}")
print(f"  F1 Score:  {train_f1:.4f}")

# Test set metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)
test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')

print("\nTEST SET:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1 Score:  {test_f1:.4f}")

# ROC AUC
try:
    train_auc = roc_auc_score(y_train, y_pred_proba_train)
    test_auc = roc_auc_score(y_test, y_pred_proba_test)
    print(f"\n  Train ROC AUC: {train_auc:.4f}")
    print(f"  Test ROC AUC:  {test_auc:.4f}")
except:
    print("\n  ROC AUC: Unable to calculate")


In [ ]:
# Detailed classification report
print("\n" + "=" * 60)
print("CLASSIFICATION REPORT (Test Set)")
print("=" * 60)
print("\nClass 0 = Negative PR")
print("Class 1 = Positive PR\n")
print(classification_report(y_test, y_pred_test, target_names=['Negative PR', 'Positive PR']))


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative PR', 'Positive PR'],
            yticklabels=['Negative PR', 'Positive PR'])
ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix - PR Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Confusion Matrix:")
print(f"  True Negatives:  {cm[0,0]}")
print(f"  False Positives: {cm[0,1]}")
print(f"  False Negatives: {cm[1,0]}")
print(f"  True Positives:  {cm[1,1]}")


In [ ]:
# Feature importance analysis
print("\n" + "=" * 60)
print("FEATURE IMPORTANCE")
print("=" * 60)

# Get feature importances
feature_importance = xgb_model.feature_importances_

# Create feature names
embedding_features = [f'embedding_{i}' for i in range(embeddings_array.shape[1])]
all_feature_names = embedding_features + available_features

# Create DataFrame for analysis
importance_df = pd.DataFrame({
    'feature': all_feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Show top metadata features (exclude embeddings for clarity)
metadata_importance = importance_df[importance_df['feature'].isin(available_features)]
print("\nTop 15 Metadata Features:")
print(metadata_importance.head(15).to_string(index=False))

# Aggregate embedding importance
embedding_importance_sum = importance_df[importance_df['feature'].str.startswith('embedding_')]['importance'].sum()
print(f"\nTotal Embedding Importance: {embedding_importance_sum:.4f}")
print(f"Total Metadata Importance: {metadata_importance['importance'].sum():.4f}")


In [ ]:
# Visualize top features
fig, ax = plt.subplots(figsize=(10, 8))

# Top 20 features overall
top_features = importance_df.head(20)
sns.barplot(data=top_features, y='feature', x='importance', ax=ax, palette='viridis')
ax.set_xlabel('Importance Score', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
ax.set_title('Top 20 Most Important Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Visualize metadata features only
if len(metadata_importance) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    top_metadata = metadata_importance.head(15)
    sns.barplot(data=top_metadata, y='feature', x='importance', ax=ax, palette='coolwarm')
    ax.set_xlabel('Importance Score', fontsize=12)
    ax.set_ylabel('Feature', fontsize=12)
    ax.set_title('Top Metadata Features by Importance', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


In [ ]:
# Sample predictions with explanations
print("\n" + "=" * 60)
print("SAMPLE PREDICTIONS")
print("=" * 60)

# Get some test samples
sample_indices = np.random.choice(len(X_test), size=min(5, len(X_test)), replace=False)

for i, idx in enumerate(sample_indices):
    original_idx = y_test.index[idx] if hasattr(y_test, 'index') else idx
    
    # Get the post data (need to match back to original dataframe)
    # This is a bit tricky since we've done train/test split
    test_indices = np.arange(len(y))[np.isin(np.arange(len(y)), 
                                               np.where(np.isin(y, y_test))[0])]
    
    print(f"\n{'='*60}")
    print(f"Sample {i+1}")
    print(f"{'='*60}")
    print(f"True Label:      {'Positive PR' if y_test[idx] == 1 else 'Negative PR'}")
    print(f"Predicted Label: {'Positive PR' if y_pred_test[idx] == 1 else 'Negative PR'}")
    print(f"Confidence:      {y_pred_proba_test[idx]:.2%}")
    print(f"Correct:         {'✓' if y_test[idx] == y_pred_test[idx] else '✗'}")


In [ ]:
# Model performance summary
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"\n✓ Model Type: XGBoost Classifier")
print(f"✓ Features: {X_combined.shape[1]} total ({embeddings_array.shape[1]} embeddings + {len(available_features)} metadata)")
print(f"✓ Training samples: {len(X_train)}")
print(f"✓ Test samples: {len(X_test)}")
print(f"\n✓ Test Accuracy: {test_accuracy:.2%}")
print(f"✓ Test F1 Score: {test_f1:.2%}")
print(f"✓ Test Precision: {test_precision:.2%}")
print(f"✓ Test Recall: {test_recall:.2%}")

print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("\n1. The model successfully classifies posts as positive or negative PR")
print("2. Gemini embeddings capture semantic content of posts")
print("3. Metadata features (engagement, sentiment, media type) provide additional signal")
print(f"4. Top predictive features include: {', '.join(metadata_importance.head(3)['feature'].values)}")
print("\n5. Use this model to:")
print("   - Predict PR impact before posting")
print("   - Identify what makes content resonate positively")
print("   - Optimize content strategy for better PR outcomes")


## Next Steps and Usage

To use this model for new posts:

```python
# 1. Generate embedding for new post text
new_post_text = "Your new post text here..."
new_embedding = get_gemini_embedding(new_post_text)

# 2. Extract metadata features (text_length, emoji_count, etc.)
# 3. Combine embedding + metadata
# 4. Scale using saved scaler
# 5. Predict: xgb_model.predict(scaled_features)
```

Model files saved:
- `pr_classifier_model.pkl` - Trained XGBoost model
- `feature_scaler.pkl` - Feature scaler
- `post_embeddings.npy` - Cached embeddings
- Encoders for categorical features
